In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax_smi
jax_smi.initialise_tracking()

In [3]:
from saex.trainer_cache import main
main()

/home/neverix/.cache/pypoetry/virtualenvs/saex-U2at97x7-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping buffer creation because buffer_max_samples < sae_config.batch_size
Creating SAE...
